In [1]:
!pip install pyspark

from google.colab import drive
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType, StructType
from pyspark.sql import Window

import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=f02a59412f706a0ec63e0f938809c651241acfb3f8f0a24ee1852059b5e1562f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# tworzenie sesji w Sparku
spark = SparkSession.builder.appName('SparkWindows').getOrCreate()

In [3]:
# wczytanie danych z google drive
drive.mount('/content/drive')

columns = ['lon', 'lat', 'Date', 'Rainf', 'Evap', 'AvgSurfT', 'Albedo','SoilT_10_40cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilM_100_200cm']

# Utworzenie schematu określającego typ zmiennych
schema = StructType()
for i in columns:
  if i == "Date":
    schema = schema.add(i, IntegerType(), True)
  else:
    schema = schema.add(i, FloatType(), True)

nasa = spark.read.format('csv').option("header", True).schema(schema).load('/content/drive/MyDrive/BigMess/NASA/NASA.csv')
nasa.show(5)

Mounted at /content/drive
+---------+-------+------+-----+---------+---------+---------+-------------+----------+---------+----------+---------------+
|      lon|    lat|  Date|Rainf|     Evap| AvgSurfT|   Albedo|SoilT_10_40cm|      GVEG|  PotEvap| RootMoist|SoilM_100_200cm|
+---------+-------+------+-----+---------+---------+---------+-------------+----------+---------+----------+---------------+
|-112.0625|25.0625|200001|  0.0|   4.3807| 288.0707| 41.47715|    289.00714|0.19712792|139.13737|  243.2525|      108.76931|
|-111.9375|25.0625|200001|  0.0|4.6673994|287.39276|41.509407|     288.8017|0.19860405|162.25638| 220.77466|       90.67495|
|-111.8125|25.0625|200001|  0.0|5.8487973| 287.6554|41.505375|    289.55984|0.17118543|121.55404| 103.95005|      161.94794|
|-111.6875|25.0625|200001|  0.0|6.4366016| 287.5386|41.501343|    289.61142|0.17118543|127.63407|106.032845|      163.44402|
|-111.5625|25.0625|200001|  0.0|3.4506986| 287.2394|41.509407|     289.2371| 0.1429876|179.37668| 1

In [4]:
# rozdzielenie kolumny Date na Year i Month
nasa.createOrReplaceTempView("nasa")
nasa = spark.sql("""
          SELECT
          CAST(SUBSTRING(CAST(Date AS STRING), 1, 4) AS INT) AS Year,
          CAST(SUBSTRING(CAST(Date AS STRING), 5, 2) AS INT) AS Month,
          n.*
          FROM nasa n
          """)

nasa = nasa.drop("Date")
nasa.show(5)

+----+-----+---------+-------+-----+---------+---------+---------+-------------+----------+---------+----------+---------------+
|Year|Month|      lon|    lat|Rainf|     Evap| AvgSurfT|   Albedo|SoilT_10_40cm|      GVEG|  PotEvap| RootMoist|SoilM_100_200cm|
+----+-----+---------+-------+-----+---------+---------+---------+-------------+----------+---------+----------+---------------+
|2000|    1|-112.0625|25.0625|  0.0|   4.3807| 288.0707| 41.47715|    289.00714|0.19712792|139.13737|  243.2525|      108.76931|
|2000|    1|-111.9375|25.0625|  0.0|4.6673994|287.39276|41.509407|     288.8017|0.19860405|162.25638| 220.77466|       90.67495|
|2000|    1|-111.8125|25.0625|  0.0|5.8487973| 287.6554|41.505375|    289.55984|0.17118543|121.55404| 103.95005|      161.94794|
|2000|    1|-111.6875|25.0625|  0.0|6.4366016| 287.5386|41.501343|    289.61142|0.17118543|127.63407|106.032845|      163.44402|
|2000|    1|-111.5625|25.0625|  0.0|3.4506986| 287.2394|41.509407|     289.2371| 0.1429876|179.37

In [20]:
# Funkcja do tworzenia cech czasowych z wykorzsytaniem funkcji window w Sparku
def overYearStats(column: str, n: int):
  """
    Funkcja liczaca statystyki minimum, maximum, srednia i mediane dla wybranej zmiennej z zakresu n miesiecy wstecz i n miesiecy w przod dla danych wspolrzednych geograficznych

    :param column: zmienna, dla ktorej liczymy min, max itd.
    :param n: liczba dni w przod i w tyl z ktorych liczymy statystyki
  """
  windowSpec = Window.partitionBy("lon", "lat").orderBy("Year", "Month").rowsBetween(-n, n)

  nasa_window = (nasa.withColumn("average_" + column, F.avg(F.col(column)).over(windowSpec))
                .withColumn("min_" + column, F.min(F.col(column)).over(windowSpec))
                .withColumn("max_" + column, F.max(F.col(column)).over(windowSpec))
                .withColumn("median_" + column, F.expr("percentile_approx(" + column + ", 0.5)").over(windowSpec))
                )
  return nasa_window

In [21]:
result = overYearStats("Evap", 5)

In [22]:
result.show(5)

+----+-----+---------+-------+----------+---------+---------+---------+-------------+----------+---------+---------+---------------+------------------+---------+---------+-----------+
|Year|Month|      lon|    lat|     Rainf|     Evap| AvgSurfT|   Albedo|SoilT_10_40cm|      GVEG|  PotEvap|RootMoist|SoilM_100_200cm|      average_Evap| min_Evap| max_Evap|median_Evap|
+----+-----+---------+-------+----------+---------+---------+---------+-------------+----------+---------+---------+---------------+------------------+---------+---------+-----------+
|1979|    1|-124.5625|47.9375| 102.24292|10.942404|274.84088| 19.94993|    277.61047|0.51115257|37.040825| 632.6485|      323.77112| 31.97905158996582|10.942404|45.125782|  30.352905|
|1979|    2|-124.5625|47.9375| 512.05865|30.025711|277.54193| 19.33631|     277.1021|0.52497137|48.730946| 684.1061|      347.15833|33.174666268484934|10.942404|45.125782|  30.520498|
|1979|    3|-124.5625|47.9375|  186.3494|30.520498| 279.1123|18.885754|     278.